# The Ovals problem

**Prompt for search setup**

Act as a research mathematician and an optimization specialst.

For a given natural number n, your task is to find three one-dimensional
python lists x, y and phi, each of which consists of n real numbers, so that the
following goal is optimized.

GOAL:
By interpolating the values x, y and phi via B-splines on the grid
np.linspace(0, 2*np.pi, n) we obtain smooth periodic one-dimensional functions
f, g and h on the interval $[0, 2 \pi]$. We consider the 2D curve $t \mapsto
f(t), g(t)$ and reparametrize by arc-length $s$ which we denote by $c(s) = (f(t(s)), g(t(s))$. We form the Rayleigh quotient given by:

$I(f, g, h) := \int_0^{2\pi} h'(t)^2 + (\kappa(t) * h(t))^2 dt / \int_0^{2\pi} h(t)^2 dt$.

Here $\kappa(s) = \|c''(s)\|$ is the curvature of the 2d curve c(s).

Your goal is to propose x, y, phi so that the quantity $I(f, g, h)$ is as
small as possible.

IMPORTANT CONSTRAINTS:
The curvature $\kappa(t)$ must be positive - in other words, you
should make sure that the 2d curve $t \mapsto (f(t), g(t))$ is convex.

SCALES:
We pose the restriction that f, g, h and their derivatives do not assume too
large values. In other words consider proposing x, y, phi that might lead to
tame B-splines. Furthermore, consider arranging that np.abs(x[0] - x[-1]),
np.abs(y[0] - y[-1]) and np.abs(phi[0] - phi[-1]) are not too large so that
when enforcing periodic boundary conditions, the splines f, g, h will not have
high derivatives and spikes near 0 and $2 \pi$.

Specifically, the Python function you have to provide has the following
signature:

def get_values(n: int) -> list[list[float] | np.ndarray]:

The function get_values(n) returns a list [x, y, phi] consisting of the
one-dimensional real arrays mentioned above.

EVALUATION:

The score function your construction will be evaluated on is given below:

def get_score(
    x1: np.ndarray | list[float],
    x2: np.ndarray | list[float],
    phi: np.ndarray | list[float],
) -> float:

The scoring function outputs I(x1, x2, phi) as a score. The smaller the score, the better your proposed configuration is.

You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have access to it,
you don't need to code up the get_score() function.
You want the score it gives you to be as small as possible!

Your task is to write a search function that searches for the best list.
Your function will have 1000 seconds to run, and after that it has to have
returned the best construction it found. If after 1000 seconds it has not
returned anything, it will be terminated with negative infinity points. You can
use your time best if you have an outer loop of the form
"while time.time() - start_time < 1000:" or similar, just don't forget to define
the "start_time" variable early in your program.

In [ ]:
# @title Examples of initial programs


def get_values(
    n: int,
) -> list[
    list[float] | np.ndarray, list[float] | np.ndarray, list[float] | np.ndarray
]:

  return (
      list(np.cos(np.linspace(0, 2 * np.pi, 2 * j + 1))[:-1]),
      list(np.sin(np.linspace(0, 2 * np.pi, 2 * j + 1))[:-1]),
      list(np.ones(2 * j + 1)),
  )



def get_values(
    n: int,
) -> list[
    list[float] | np.ndarray, list[float] | np.ndarray, list[float] | np.ndarray
]:
  """Generate functions to optimize ratio."""
  variable_name = f'suggested_values_{n}'
  best_values = [
      list(np.linspace(0, 2 * np.pi, n + 1)[:-1] ** 2 + 1.234),
      list(np.linspace(0, 2 * np.pi, n + 1)[:-1] - 2.345),
      list(np.linspace(0, 2 * np.pi, n + 1)[:-1] - 3.456),
  ]

  if np.random.rand() < 0.5 and variable_name in globals():
    best_values = globals()[variable_name]

  curr_values = best_values.copy()
  best_score = get_score(*best_values)

  start_time = time.time()

  while time.time() - start_time < 100:  # Search for 100 seconds
    # Mutate best construction
    random_index = np.random.randint(0, len(curr_values))
    curr_values[random_index:] += 1e-1 * np.random.randint(-10, 10)
    curr_score = get_score(n, curr_values)
    if curr_score  best_score:
      best_score = curr_score
      best_values = curr_values.copy()
      print(f"Best score: {curr_score}")
  return best_values

In [ ]:
# @title Evaluation

from collections.abc import Callable
import dataclasses
import enum
from typing import Any, List, Tuple
import jax
import jax.numpy as jnp
import numpy as np
import scipy



def get_quadrature_points_and_weights(grid: np.ndarray, order: int) -> Any:
  """Generates sample points and weights for Gaussian quadrature integration."""
  diff = grid[1:] - grid[:-1]
  sample_points_pattern = (np.polynomial.legendre.leggauss(order)[0] + 1) / 2
  weights_pattern = np.polynomial.legendre.leggauss(order)[1] / 2

  sample_points = (
      np.repeat(grid[:-1][:, None], order, axis=-1)
      + np.repeat(diff[:, None], order, axis=-1) * sample_points_pattern
  )

  weights = np.repeat(diff[:, None], order, axis=-1) * weights_pattern
  return sample_points, weights


def get_cumulative_integration_fn(
    func: Callable[[np.ndarray], np.ndarray],
    order: int = 4,
    add_zero: bool = False,
) -> Callable[[np.ndarray], np.ndarray]:
  """Returns a cumulative integration function for a given integrand."""

  def int_func(grid: np.ndarray) -> np.ndarray:
    sample_points, weights = get_quadrature_points_and_weights(grid, order)
    func_samples = func(sample_points.flatten()).reshape(sample_points.shape)
    cumulative_sum = np.cumsum(np.sum(func_samples * weights, axis=1))

    if add_zero:
      return np.append(np.zeros(()), cumulative_sum)
    else:
      return cumulative_sum

  return int_func


def get_scipy_spline(f_vals: np.ndarray | list[float], spline_order: int):
  num_points = len(f_vals)
  f_vals[-1] = f_vals[0]
  xs = np.linspace(0, 2 * np.pi, num_points)
  scipy_spline = scipy.interpolate.make_interp_spline(
      xs, f_vals, k=spline_order, bc_type="periodic"
  )
  return scipy_spline


# Parametrization endpoints
T_START = 0.0
T_END = 2 * np.pi

NEGATIVE_INFINITY = -1e10

NUM_CURVE_SAMPLES = 10000
SPEED_DEGENERACY_THRESHOLD = 1e-4
CURVE_QUADRATURE_PRECISION_THRESHOLD = 1e-5
TOTAL_ARCLENGTH_DEGENERACY_THRESHOLD = 1e-3
PHI_DEGENERACY_THRESHOLD = 1e-3
PHI_QUADRATURE_PRECISION_THRESHOLD = 1e-5
LOW_QUADRATURE_PRECISION = 1002


class ErrorMessage(enum.Enum):
  """Error messages for the curve evaluation."""

  NO_ERROR = 0
  DEGENERATE_CURVE_SPEED = 1
  DEGENERATE_PHI = 2
  LOW_QUADRATURE_PRECISION = 3
  WRONG_OUTPUT_SHAPE = 4
  FUNCTION_IS_TOO_LARGE = 5
  ZERO_FUNCTION_SCORE = 6
  STOCHASTIC_FUNCTION = 7
  POINTS_NOT_FLOATS = 8
  NEGATIVE_CURVATURE = 9
  DEGENERATE_CURVE_LENGTH = 10
  OTHER_ERROR = 11


@dataclasses.dataclass(frozen=True)
class CurveEvaluationLog:
  score: float
  error_message: ErrorMessage


def evaluate_curve(
    x1: np.ndarray | list[float],
    x2: np.ndarray | list[float],
    phi: np.ndarray | list[float],
) -> CurveEvaluationLog:
  """Returns the score of the given curve."""
  x1s = list(x1.copy())
  x2s = list(x2.copy())
  phis = list(phi.copy())

  # Add starting point at the end for periodicity spline interpolation
  x1s.append(x1s[0])
  x2s.append(x2s[0])
  phis.append(phis[0])
  x1_fn = get_scipy_spline(x1s, 4)
  x2_fn = get_scipy_spline(x2s, 4)
  phi_fn = get_scipy_spline(phis, 4)

  d1x1 = x1_fn.derivative(1)
  d1x2 = x2_fn.derivative(1)
  dx_norm_fn = lambda x: np.sqrt(d1x1(x) ** 2 + d1x2(x) ** 2)

  # Check curve degeneracy
  xs = np.linspace(T_START, T_END, NUM_CURVE_SAMPLES)
  dx_norms = dx_norm_fn(xs)

  if np.min(dx_norms) < SPEED_DEGENERACY_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.DEGENERATE_CURVE_SPEED
    )

  total_arc_length = scipy.integrate.quad(
      dx_norm_fn,
      T_START,
      T_END,
      epsrel=1e-14,
      epsabs=1e-14,
      limit=10000,
  )

  if total_arc_length[0] < TOTAL_ARCLENGTH_DEGENERACY_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.DEGENERATE_CURVE_LENGTH
    )

  if total_arc_length[1] > CURVE_QUADRATURE_PRECISION_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.LOW_QUADRATURE_PRECISION
    )

  print("Total arc length:", total_arc_length)

  phi_l2_norm = scipy.integrate.quad(
      lambda x: phi_fn(x) ** 2,
      0,
      2 * np.pi,
      epsrel=1e-14,
      epsabs=1e-14,
      limit=10000,
  )

  if phi_l2_norm[0] < PHI_DEGENERACY_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.DEGENERATE_PHI
    )

  if phi_l2_norm[1] > PHI_QUADRATURE_PRECISION_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.LOW_QUADRATURE_PRECISION
    )

  # Rescale to ensure total arc-length is 2*pi
  x1_fn.c = x1_fn.c * 2 * np.pi / total_arc_length[0]
  x2_fn.c = x2_fn.c * 2 * np.pi / total_arc_length[0]

  d1x1_fn = x1_fn.derivative(1)
  d2x1 = x1_fn.derivative(2)

  d1x2_fn = x2_fn.derivative(1)
  d2x2 = x2_fn.derivative(2)

  dx_norm_fn = lambda t: np.sqrt(d1x1_fn(t) ** 2 + d1x2_fn(t) ** 2)

  # Prepare natural parametrization
  arclength_fn = get_cumulative_integration_fn(dx_norm_fn)
  t_samples = np.linspace(T_START, T_END, NUM_CURVE_SAMPLES)
  s_samples = arclength_fn(t_samples)

  t_of_s = scipy.interpolate.make_interp_spline(
      s_samples,
      t_samples[1:],
      k=4,
  )

  d1t_of_s = t_of_s.derivative(1)
  d2t_of_s = t_of_s.derivative(2)

  s_vals = np.linspace(0, 2 * np.pi, NUM_CURVE_SAMPLES)

  d1c1_fn = lambda s: d1x1(t_of_s(s)) * d1t_of_s(s)
  d1c2_fn = lambda s: d1x2(t_of_s(s)) * d1t_of_s(s)

  d2c1_fn = lambda s: d1x1(t_of_s(s)) * d2t_of_s(s) + d2x1(
      t_of_s(s)
  ) * d1t_of_s(s)

  d2c2_fn = lambda s: d1x2(t_of_s(s)) * d2t_of_s(s) + d2x2(
      t_of_s(s)
  ) * d1t_of_s(s)

  d1phi = phi_fn.derivative(1)

  # Compute curvature
  kappa = lambda s: np.sqrt(d2c1_fn(s) ** 2 + d2c2_fn(s) ** 2)
  signed_kappa = lambda s: (d1c1_fn(s) * d2c2_fn(s) - d2c1_fn(s) * d1c2_fn(s))

  kappas = kappa(s_vals)
  signed_kappas = signed_kappa(s_vals)

  if np.min(kappas) < 0.0 or np.min(signed_kappas) < 0.0:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.NEGATIVE_CURVATURE
    )

  i_x_phi = scipy.integrate.quad(
      lambda s: (d1phi(s) ** 2 + (kappa(s) * phi_fn(s)) ** 2),
      0,
      2 * np.pi,
      epsrel=1e-14,
      epsabs=1e-14,
      limit=NUM_CURVE_SAMPLES,
  )

  if i_x_phi[1] > CURVE_QUADRATURE_PRECISION_THRESHOLD:
    return CurveEvaluationLog(
        score=0.0, error_message=ErrorMessage.LOW_QUADRATURE_PRECISION
    )

  return CurveEvaluationLog(
      score=i_x_phi[0] / phi_l2_norm[0], error_message=ErrorMessage.NO_ERROR
  )


def get_score(
    x1: np.ndarray | list[float],
    x2: np.ndarray | list[float],
    phi: np.ndarray | list[float],
) -> float:
  """Computes the area of a given configuration of triangles."""
  curve_eval_log = evaluate_curve(x1, x2, phi)
  if curve_eval_log.error_message != ErrorMessage.NO_ERROR:
    return NEGATIVE_INFINITY
  return -curve_eval_log.score


In [ ]:
# @title Examples of programs evolved by AlphaEvolve

def get_values(
    n: int,
) -> list[
    list[float] | np.ndarray, list[float] | np.ndarray, list[float] | np.ndarray
]:
  """Generate functions to optimize ratio."""

  theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
  r = 1.0
  x = r * np.cos(theta)
  y = r * np.sin(theta)
  phi = np.ones_like(theta)

  start_time = time.time()
  best_score = float('inf')
  best_x = list(x)
  best_y = list(y)
  best_phi = list(phi)

  while time.time() - start_time < 99: #Run search for this many seconds

    # Perturb x, y, phi
    # CRAZY IDEA: Introduce a "gravity" towards a simpler solution (circle, constant phi),
    # but also add a random impulse every few iterations to escape local minima.

    gravity = 0.001 # Strength of attraction to initial state.
    impulse_frequency = 10 # Apply impulse every this many iterations

    x_perturbed = x + 0.01 * np.random.randn(n) * (time.time() - start_time)/100 - gravity * (x - np.cos(theta))
    y_perturbed = y + 0.01 * np.random.randn(n) * (time.time() - start_time)/100 - gravity * (y - np.sin(theta))
    phi_perturbed = phi + 0.01 * np.random.randn(n) * (time.time() - start_time)/100 - gravity * (phi - np.ones_like(theta))

    #if int(time.time() * 10) % impulse_frequency == 0:  # Apply impulse
    #  x_perturbed += 0.1 * np.random.randn(n)
    #  y_perturbed += 0.1 * np.random.randn(n)
    #  phi_perturbed += 0.1 * np.random.randn(n)


    try:
      score = get_score(list(x_perturbed), list(y_perturbed), list(phi_perturbed))

      if score < best_score and score > 0: #Score needs to be positive.
        best_score = score
        best_x = list(x_perturbed)
        best_y = list(y_perturbed)
        best_phi = list(phi_perturbed)
        print(f"New best score: {best_score}")
    except Exception as e:
      #print(f"Error: {e}")
      pass

  return [best_x, best_y, best_phi]